# Etapa 01 – Coleta dos Dados
### Equipe: Ana Carolina Estrela, Eduardo Ikeda, Vinicius Maciel
### Estado: Pernambuco

A extração inicial dos dados foi feita pelo proprio site do Datasus e sua extração foi feita utilizando o tabwin. As fases subsequentes foram feitas utilizando a biblioteca Pandas do python


In [1]:
import pandas as pd

### 4.a – Quantos registros de AIH existem para cada mês e em cada estado?

In [2]:
chunksize = 100_000  
resultados = {}

for chunk in pd.read_csv("combined_data.csv", sep=",", encoding="latin1", chunksize=chunksize, low_memory=False):
    chunk.columns = chunk.columns.str.strip()
    
    # Contar registros por ano e mês
    contagem = chunk.groupby(["ANO_CMPT", "MES_CMPT"]).size()
    
    for idx, valor in contagem.items():
        resultados[idx] = resultados.get(idx, 0) + valor

df_resultado = pd.DataFrame(list(resultados.items()), columns=["(ano, mes)", "quantidade"])
df_resultado[["ano", "mes"]] = pd.DataFrame(df_resultado["(ano, mes)"].tolist(), index=df_resultado.index)
df_resultado = df_resultado.drop(columns="(ano, mes)")

df_resultado = df_resultado.sort_values(["ano", "mes"])
print(df_resultado)
df_resultado.to_csv("4a_resultado_mes_ano.csv", index=False, encoding="utf-8")




    quantidade   ano  mes
0        46340  2019    1
1        44356  2019    2
2        47317  2019    3
3        49156  2019    4
4        51443  2019    5
..         ...   ...  ...
67       59288  2024    8
68       57731  2024    9
69       58387  2024   10
70       53576  2024   11
71       53957  2024   12

[72 rows x 3 columns]


In [4]:
resultados = {} 

for chunk in pd.read_csv("combined_data.csv", sep=",", encoding="latin1", chunksize=chunksize, low_memory=False):
    chunk.columns = chunk.columns.str.strip()
    
    contagem = chunk.groupby("ANO_CMPT").size()
    
    for idx, valor in contagem.items():
        resultados[idx] = resultados.get(idx, 0) + valor

df_resultado = pd.DataFrame(list(resultados.items()), columns=["ano", "quantidade"])

df_resultado = df_resultado.sort_values("ano").reset_index(drop=True)
print(df_resultado)
df_resultado.to_csv("4a_resultado_ano.csv", index=False, encoding="utf-8")


    ano  quantidade
0  2019      635467
1  2020      490539
2  2021      557344
3  2022      599329
4  2023      617948
5  2024      669715


### 4.b – Quantos registros de Procedimentos de Saúde foram importados?

In [5]:
total_procedimentos = 0

for chunk in pd.read_csv("combined_data.csv", sep=",", encoding="latin1", chunksize=chunksize, low_memory=False):
    chunk.columns = chunk.columns.str.strip()
    
    
    total_procedimentos += chunk["ESPEC"].notna().sum()

print("Total de registros de procedimentos:", total_procedimentos)



Total de registros de procedimentos: 3570342


### 4.c – Quantos registros de CID foram importados?


In [6]:
total_cid  = 0

for chunk in pd.read_csv("combined_data.csv", sep=",", encoding="latin1", chunksize=chunksize, low_memory=False):
    chunk.columns = chunk.columns.str.strip()

    cid_presente = chunk[['CID_NOTIF', 'CID_ASSO', 'CID_MORTE']].notna().any(axis=1)
    total_cid += cid_presente.sum()

print("Total de registros com pelo menos um CID preenchido:", total_cid)

Total de registros com pelo menos um CID preenchido: 3570342


### 4.d – Quantos registros de estabelecimentos (CNES) foram importados?

In [ ]:
total_estabelecimentos = 0

for chunk in pd.read_csv("combined_data.csv", sep=",", encoding="latin1", chunksize=chunksize, low_memory=False):
    chunk.columns = chunk.columns.str.strip()
    
    registros_validos = pd.to_numeric(chunk['CNES'], errors='coerce').notna()
    
    total_estabelecimentos += registros_validos.sum()

print("Total de registros de estabelecimentos válidos:", total_estabelecimentos)

Total de registros de estabelecimentos válidos: 3570342
